In [4]:
# 查看无效的 place pulse 图片列表
# 比如 514135dcfdc9f04926004b61
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
image_dir = 'F:\\data\\PlacePulse\\images'
files = os.listdir(image_dir)
sample_image_file_name = os.path.join(image_dir, '514135dcfdc9f04926004b61.jpg')
sample_image = cv2.imread(sample_image_file_name)
plt.imshow(sample_image)
broken_list = []
cnt = 0
for file in tqdm(files):
    cur_img = cv2.imread(os.path.join(image_dir, file))
    if np.max(np.abs(cur_img - sample_image)) == 0:
        broken_list.append(file)
    cnt += 1

100%|██████████| 111390/111390 [08:01<00:00, 231.47it/s]


OSError: [Errno 22] Invalid argument: 'E:\x0bscprojects\\pytorch-relative-attributes\\logs\\borken_img_list_place_pulse.txt'

In [6]:
# 共 663 张无效图片，下面保存无效图片名称
with open('E:\\vscprojects\\pytorch-relative-attributes\\logs\\borken_img_list_place_pulse.txt', 'w') as f:
    for line in broken_list:
        f.write(line + '\n')

In [9]:
# 通过 broken_list_file 获取所有受影响的图片列表
# 更新csv
import os
import pandas as pd
src_csv_file = os.path.join("F:\\data\\PlacePulse\\votes.csv")
target_csv_file = os.path.join("F:\\data\\PlacePulse\\annos.csv")
broken_names = [name[:name.find('.')] for name in broken_list]
df = pd.read_csv(src_csv_file)

In [10]:
delete_line_ids = []
cur_line = 0
for r in df.iterrows():
    if r[1].left_id in broken_names or r[1].right_id in broken_names:
        delete_line_ids.append(cur_line)
    cur_line += 1
print('{} has {} error samples'.format(src_csv_file, len(delete_line_ids)))
df = df.drop(delete_line_ids)
df.to_csv(target_csv_file, index=False)

F:\data\PlacePulse\votes.csv has 14093 error samples
